## Agents

Agents use an LLM to determine which actions to perform and in what order. An action can be either using a tool and observing its output or returning it to the user. To use an agent, in addition to the concept of an LLM, it is important to understand a new concept and that of a "tool".

### Tools

Tools are functions that agents can use to interact with the world. These tools can be common utilities (e.g. search), other chains, or even other agents.

In [16]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [17]:
from langchain.agents import load_tools
from langchain_openai import ChatOpenAI
from langchain.agents import AgentType

llm = ChatOpenAI(model="gpt-4o-mini")

tool_names = ["llm-math"]
tools = load_tools(tool_names, llm=llm)
tools

[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question

In [18]:
from langchain.agents import initialize_agent

agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         max_iterations=3)

Lets have a look at the ReAct template

In [19]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Calculator(*args: Any, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[list[str]] = None, metadata: Optional[dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [20]:
agent.invoke(input="How many members does the A Team have?")



> Entering new AgentExecutor chain...
To answer the question about how many members the A Team has, I need to consider the context of "A Team." This could refer to various groups, such as a sports team, a fictional team from a TV show, or a specialized group in a particular industry. I need to clarify which "A Team" is being referenced to provide an accurate answer. 

In popular culture, "The A-Team" refers to a fictional group of ex-soldiers who work together on missions to help people, famously depicted in the 1980s television series. The main characters include four members: John "Hannibal" Smith, Templeton "Faceman" Peck, B.A. Baracus, and H.M. Murdock, which strictly makes it four members.

Final Answer: The A Team has 4 members.

> Finished chain.


{'input': 'How many members does the A Team have?',
 'output': 'The A Team has 4 members.'}

In [21]:
agent.invoke(input="What is 100 devided by 25?")



> Entering new AgentExecutor chain...
To solve the question, I need to perform the division of 100 by 25. 

Action: Calculator
Action Input: 100 / 25
Observation: Answer: 4.0
Thought:I now know the final answer.  
Final Answer: 4.0

> Finished chain.


{'input': 'What is 100 devided by 25?', 'output': '4.0'}

### Custom Tools

You can also create your own tools by creating a class that inherits from BaseTool.

In [22]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings()

vectorstore = FAISS.load_local("index", embeddings, allow_dangerous_deserialization=True)

In [23]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun

class CustomSearchTool(BaseTool):
    name: str = "restaurant search"
    description: str = "useful for when you need to answer questions about our restaurant"

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        store = vectorstore.as_retriever()
        docs = store.invoke(query)
        text_list = [doc.page_content for doc in docs]
        return "\n".join(text_list)

    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [24]:
from langchain.agents import AgentType

tools = [CustomSearchTool()]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, max_iterations=3)

In [26]:
agent.invoke(input="Who are the members of the A-Team?")



> Entering new AgentExecutor chain...
The A-Team is a fictional group from the television series and movie of the same name. The main members include:

1. John "Hannibal" Smith - the leader of the team.
2. Templeton "Face" Peck - the smooth-talking conman.
3. B.A. Baracus - the strong and tough mechanic.
4. H.M. "Howlin' Mad" Murdock - the eccentric pilot.

Thought: I have provided the necessary information about the members of the A-Team.
Final Answer: The members of the A-Team are John "Hannibal" Smith, Templeton "Face" Peck, B.A. Baracus, and H.M. "Howlin' Mad" Murdock.

> Finished chain.


{'input': 'Who are the members of the A-Team?',
 'output': 'The members of the A-Team are John "Hannibal" Smith, Templeton "Face" Peck, B.A. Baracus, and H.M. "Howlin\' Mad" Murdock.'}

In [29]:
agent.invoke(input="When does the restaurant open?")



> Entering new AgentExecutor chain...
I need to find out the opening hours of the restaurant.  
Action: restaurant search  
Action Input: "restaurant hours"  
Observation: during weekends and holidays, to ensure a seamless dining experience.
A: While walk-ins are always welcome, we recommend making a reservation, especially during weekends
menu and offer high chairs and booster seats for our younger guests.
dinner or a casual meal with friends, Bella Vista provides the perfect atmosphere.
Thought:It seems that the observation provided does not include specific opening hours. I need to gather more precise information about the restaurant's opening hours to answer the question accurately.  

Action: restaurant search  
Action Input: "restaurant opening hours"  
Observation: during weekends and holidays, to ensure a seamless dining experience.
Sundays, we welcome guests from 12 p.m. to 10 p.m.
A: While walk-ins are always welcome, we recommend making a reservation, especially during wee

{'input': 'When does the restaurant open?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

### Chat Agents

In [30]:
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm = ChatOpenAI(model="gpt-4o-mini")
agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

/var/folders/9l/6q0d47xn7p70rnjn04w1vvyw0000gn/T/ipykernel_39236/2151239754.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [31]:
print(agent_chain.agent.llm_chain.prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations a

In [32]:
agent_chain.invoke(input="What is the capital of France?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "The capital of France is Paris."
}
```

> Finished chain.


{'input': 'What is the capital of France?',
 'chat_history': [HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={})],
 'output': 'The capital of France is Paris.'}

In [33]:
agent_chain.invoke(input="Any suggestions what to visit there?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "In Paris, you might consider visiting the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, Montmartre, and the Champs-Élysées. Additionally, don't miss the beautiful gardens like the Jardin du Luxembourg and historical sites like the Palace of Versailles."
}
```

> Finished chain.


{'input': 'Any suggestions what to visit there?',
 'chat_history': [HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Any suggestions what to visit there?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="In Paris, you might consider visiting the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, Montmartre, and the Champs-Élysées. Additionally, don't miss the beautiful gardens like the Jardin du Luxembourg and historical sites like the Palace of Versailles.", additional_kwargs={}, response_metadata={})],
 'output': "In Paris, you might consider visiting the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, Montmartre, and the Champs-Élysées. Additionally, don't miss the beautiful gardens like the Jardin du Luxembourg and historical sites like the Palace of Versailles."}